# Sales Forecasting

© Explore Data Science Academy

---
<a id="top"></a>
## EDSA Internship
### Team 28
#### Project Title: Develop an Accurate Forecasting Approach to Predict Future Sales Per Item Per Store

#### Team Members
- Joshua Olalemi
- Koketso Tjale
- Shadrack Darku
- Ifeanyi Okonkwo
- Thapelo Mofokeng
---
<img src="resources/supermarket.jpg">

--- 

# Useful  Links
* [Dipanshu Rana](https://github.com/ml-projects-rana/M5-Forecasting---Accuracy)

* [Laura Fink](https://www.kaggle.com/code/allunia/m5-sales-uncertainty-prediction)

* [MARISAKA MOZZ](https://www.kaggle.com/code/marisakamozz/m5-prophet/notebook)

## Table of Contents <a id="cont"></a>



### 1. [Importing Packages](#one) <br>


### 2. [Loading Data](#two) <br>

   
   2.1 [Helper Functions](#2.1) <br>


### 3. [Exploratory Data Analysis (EDA)](#three) <br>

   
   3.1 [Non-Graphical Analysis](#3.1) <br>
   3.2 [Graphical Analysis](#3.2) <br>


### 4. [Features Engineering](#four) <br>

   
   4.1 [Melting](#4.1) <br>
   4.2 [Merging](#4.2) <br>
   4.3 [Lag Features](#4.3) <br>
   4.4 [Rolling_Mean Features](#4.4) <br>
   4.5 [Dropping Redundant Features](#4.5) <br>
   4.6 [Label Encoding](#4.6) <br>
   4.7 [Saving Engineered Features](#4.7) <br>
   4.8 [Releasing Memory](#4.8) <br>




### 5. [Modeling](#five) <br>

   
   5.1 [FacebookProphet](#5.1) <br>
   5.2 [LightGBM](#5.2) <br>
   5.4 [TensorFlow LSTM](#5.3) <br>


### 6. [Model Performance](#six) <br>


### 7. [Model Explanations](#seven) <br>


### 8. [References](#eight) <br>


### 8. [Acknowledgement ](#nine) <br>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section, we imported and briefly discussed the libraries that were used throughout our analysis and modelling. |

---

In [ ]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

#For collecting 'garbage' and releasing memory
import gc
from downcast import reduce

#for converting lists to table format
import tabletext

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#for plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode()

#data preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.neighbors import KDTree

#for saving file
import pickle

#cell timing
import time

#for progress bar visualization
from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

#Modelling
from prophet import *
from fbprophet import Prophet
from fbprophet.plot import plot_plotly

from multiprocessing import Pool


from lightgbm import LGBMRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import datetime


# Import widgets
from ipywidgets import widgets, interactive, interact
import ipywidgets as widgets
from IPython.display import display

from math import log, floor

import seaborn as sns
sns.set_style('whitegrid')

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad

import scipy
import statsmodels
from scipy import signal

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.stattools import adfuller

import itertools
from itertools import cycle
plt.style.use('seaborn')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])


<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section we loaded the given data into a `DataFrame`. |

---

<a id="2.1"></a>
### 2.1 Helper Functions

---

Due to the hugeness of the datasets used, we had to optimize the memory usage by creating some functions to convert the columns to efficient datatypes.

In [ ]:
%%time
def load_csv_optimized(path: str) -> pd.DataFrame:
    """Load the CSV and cast to more memory efficient dtypes.

    Note: This is a minimal example, you can extend this function to handle additional things like columns you want excluded, etc.
    """
    _df = pd.read_csv(
        path,
        parse_dates=True,
        infer_datetime_format=True,
        low_memory=False,
    )
    _df.convert_dtypes()

    fcols = _df.select_dtypes("float").columns
    icols = _df.select_dtypes("integer").columns
    ccols = _df.select_dtypes("object").columns

    _df[fcols] = _df[fcols].apply(pd.to_numeric, downcast="float")
    _df[icols] = _df[icols].apply(pd.to_numeric, downcast="integer")
    _df[ccols] = _df[ccols].astype("category")

    print(
        f"Successfully loaded {path} using {_df.memory_usage(index=True,deep=True).sum()/1024/1024:.2f}MB RAM"
    )

    return _df

In [ ]:
%%time
def reduce_mem_usage(df, verbose=True):
    """
    This function takes in a dataframe, changes its datatypes and returns it.
    """
    
    
    """
    return to improvise on this function based on this concept
    integers = {
                'int8':'np.iinfo(np.int8)',
                'int16': 'np.iinfo(np.int16)',
                'int32':'np.iinfo(np.int32)',
                'int64':'np.iinfo(np.int64)'
                }
    
    floats = {
            'float16':'np.ninfo(np.int16)',
            'float32':'np.finfo(np.float32)',
            'float64':'np.finfo(np.float64)'
             }
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns: #get the columns in df
        col_type = df[col].dtypes #get the datatypes of the columns
        if col_type in numerics: #numerics
            min_value = df[col].min()
            max_value = df[col].max()
            if str(col_type)[:3] == 'int':
                if min_value > np.iinfo(np.int8).min and max_value < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif min_value > np.iinfo(np.int16).min and max_value < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif min_value > np.iinfo(np.int32).min and max_value < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif min_value > np.iinfo(np.int64).min and max_value < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if min_value > np.finfo(np.float16).min and max_value < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif min_value > np.finfo(np.float32).min and max_value < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
#Load the needed datasets using the created function above
df_calendar = load_csv_optimized("~/sales_data/raw_data/calendar.csv")
df_sell_prices = load_csv_optimized("~/sales_data/raw_data/sell_prices.csv")
df_train = load_csv_optimized("~/sales_data/raw_data/sales_train_evaluation.csv")
#df_test = load_csv_optimized("~/sales_data/raw_data/sales_train_validation.csv")
#df_submission = load_csv_optimized("~/sales_data/raw_data/sample_submission.csv") # consider loading this later, you just need to know the structure if you want to submit to kaggle

In [ ]:
%%time
#Optimize the datatypes of the datasets
df_calendar = reduce_mem_usage(df_calendar)
df_sell_prices = reduce_mem_usage(df_sell_prices)
df_train = reduce_mem_usage(df_train)
#df_test = reduce_mem_usage(df_test)
#df_submission = reduce_mem_usage(df_submission)

<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, we explored and performed an in-depth analysis of all the variables in the DataFrame. |

---


The Exploratory Data Analysis (EDA) we conducted helped us to understand the data we were working with without making any assumptions. This formed a vital component before we continued with the modelling phase, as it provided context and guidance on the course of action to take when developing the appropriate model.

This was done in 2-fold, viz;
- Non-Graphical Analysis
- Graphical Analysis

<a id="3.1"></a>
### 3.1 Non-Graphical Analysis

---

#### `df_calendar`

In [ ]:
%%time
df_calendar.head()

In [ ]:
%%time
df_calendar.info()

In [ ]:
%%time
df_calendar.isnull().sum()

#### `df_sell_price`

In [ ]:
df_sell_prices.head()

In [ ]:
df_sell_prices.info()

In [ ]:
df_sell_prices.isnull().sum()

#### `df_train`

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

In [ ]:
df_train.describe()

<a id="3.2"></a>
### 3.2 Graphical Analysis

---

In [ ]:
def patch1(bar,ax):
    for p in bar.patches:
        width=p.get_width()
        height=p.get_height()
        x,y=p.get_xy()
        ax.annotate('{}%'.format(height),(x+width/2,y+height*1.02),ha='center',fontsize=14, fontweight='bold')

In [ ]:
def patch2(a,ax):
    for p in a.patches:
        width=p.get_width()
        height=p.get_height()
        x,y=p.get_xy()
        ax.annotate('{:.2f}'.format(height),(x+width/2,y+height*1.02),ha='center',fontsize=13)

In [ ]:
%%time
def days_with_event():
    l=[]
    l=np.unique(df_calendar[df_calendar['event_type_1'].notnull()]['d'].tolist())
    df=pd.DataFrame([["event",np.round((len(l)/df_calendar.shape[0]*100),2)],
                     ["no_event",100-np.round((len(l)/df_calendar.shape[0]*100),2)]],
                    columns=['Events','Percentage'])
    fig,axes=plt.subplots(figsize=(10,8))
    a = df.plot(kind='bar',y='Percentage',x='Events',width=0.4,ax=axes,color='#3b82f6',edgecolor='black')
    patch1(a,axes)
    plt.title('Days with Events/No Events',loc='center',fontsize=25,fontweight='bold')
    axes.set_xlabel('Event',fontsize=18,labelpad=15)
    axes.set_ylabel('No_of_Occurences',fontsize=18,labelpad=15)
    plt.xticks(rotation='horizontal',fontsize=14)
    plt.yticks(fontsize=14)
    axes.legend(loc='upper left')
    plt.savefig('../sales_data/notebook_resources/days_with_events')
    plt.show()
    
days_with_event()
del days_with_event

In [ ]:
%%time
def temp_merge():
    s = pd.melt(df_train,id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],
              var_name='d',
              value_name='demand')
    s = pd.merge(s,df_calendar,
               on='d',
               how='left')
    revenue = pd.merge(s,
                       df_sell_prices,
                       on=['item_id','store_id','wm_yr_wk'],
                       how='left')
    #Calculating total cost on that day (cost = no. of sales of item * sell price of item)
    revenue['cost']=revenue.demand*revenue.sell_price 
    return revenue
revenue = temp_merge()
revenue

In [ ]:
%%time
def daily_sales():
    d=revenue[['weekday','demand']]
    d=d.groupby(by='weekday').sum('demand').reset_index()
    d['percent']=np.round(d['demand']/d['demand'].sum()*100,2)
    d=d.sort_values('percent')
    fig,axes=plt.subplots(figsize=(12,8))
    a=d.plot(kind='bar',x='weekday',y='percent',width=0.6,ax=axes,color='#3b82f6', edgecolor='black')
    patch1(a,axes)
    plt.title('Sales per Day (Ascending Sort)',loc='center',fontsize=25,pad='20',fontweight='bold')
    axes.set_ylabel('Percentage',fontsize=18,labelpad=15)
    axes.set_xlabel('Weekday',fontsize=18,labelpad=15)
    plt.xticks(rotation='horizontal',fontsize=14)
    plt.yticks(fontsize=14)
    axes.get_legend().remove()
    #plt.savefig("../sales_data/notebook_resources/daily_sales")
    plt.show()

daily_sales()
del revenue, daily_sales

In [ ]:
def monthly_sales():
    revenue = temp_merge()
    d=revenue[['month','demand']]
    d=d.groupby(by='month').sum('demand').reset_index()
    d['percent']=np.round(d['demand']/d['demand'].sum()*100,2)
    fig,axes=plt.subplots(figsize=(12,8))
    a=d.plot(kind='bar',x='month',y='percent',width=0.6,ax=axes,color='#3b82f6', edgecolor='black')
    patch1(a,axes)
    plt.title('Sales per Month',loc='center',fontsize=25,pad='20',fontweight='bold')
    axes.set_ylabel('Percentage',fontsize=18,labelpad=15)
    axes.set_xlabel('Month',fontsize=18,labelpad=15)
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    axes.set_xticklabels(months)
    plt.xticks(rotation='horizontal',fontsize=14)
    plt.yticks(fontsize=14)
    axes.get_legend().remove()
    plt.savefig("../sales_data/notebook_resources/monthly_sales")
    plt.show()
    
monthly_sales()
del monthly_sales

In [ ]:
def yearly_sales():
    revenue = temp_merge()
    d=revenue[['year','demand']]
    d=d.groupby(by='year').sum('demand').reset_index()
    d['percent']=np.round(d['demand']/d['demand'].sum()*100,2)
    d=d.sort_values('year')
    fig,axes=plt.subplots(figsize=(11,8))
    a=d.plot(kind='bar',x='year',y='percent',width=0.6,ax=axes,color='#3b82f6', edgecolor='black')
    patch1(a,axes)
    plt.title('Sales per Year',loc='center',fontsize=25,pad='20',fontweight='bold')
    axes.set_ylabel('Percentage',fontsize=18,labelpad=15)
    axes.set_xlabel('Year',fontsize=18,labelpad=15)
    plt.xticks(rotation='horizontal',fontsize=14)
    plt.yticks(fontsize=14)
    axes.get_legend().remove()
    #plt.savefig("../sales_data/notebook_resources/yearly_sales")
    plt.show()
    
yearly_sales()
del yearly_sales

### Questions on distribution of product & behavior across timeline

Starting with the dataframe denoted by train_sales_df that has the item specific ('id'), locale specific ('store_id' , 'state_id') and sales days specific (d_1 to d_1913) information; let us first make necessary adjustments to separate the sales days so that analysis along item_id, store_id and dept_id can be more easily observed across sales days only. 

In [ ]:
df_train.head(3)

In [ ]:
d_cols = [c for c in df_train.columns if 'd_' in c]
df_train_ins = df_train.copy() #create an insurance copy of df_train
df_train_ins['total_sales_all_days'] = df_train_ins[d_cols].sum(axis = 1)
df_train_ins['avg_sales_all_days'] = df_train_ins[d_cols].mean(axis = 1)
df_train_ins['median_sales_all_days'] = df_train_ins[d_cols].median(axis = 1)
#train_sales_df.groupby(['id'])['total_sales_all_days'].sum().sort_values(ascending=False)

#### Distribution of product_ids across categories?

In [ ]:
df = df_train_ins.groupby(['cat_id'])['id'].count().reset_index(name='total_entries')
fig = px.pie(df, values='total_entries', names='cat_id', 
            color_discrete_sequence=px.colors.sequential.RdBu,
            width = 750, height=450, title = 'Distribution of product_IDs across categories')
fig.show()
del df, d_cols

Food items are the most sold out item that are followed by household items and then hobbies items.

In [ ]:
df = df_train_ins.groupby(['state_id'])['total_sales_all_days'].sum().reset_index()
fig = px.pie(df, values='total_sales_all_days', names='state_id', 
            color_discrete_sequence=px.colors.sequential.Aggrnyl,
            width = 750, height=450, title = 'Distribution of Total_Sales Across States')
fig.show()
del df

With respect to the total number of sales, it is evident once again that the number of items sold on total have the greatest contributing share in CA, followed by Texas and Wisconsin. Now is it the case with the total revenue generated as well? We'd find that out using the revenue dataframe 

In [ ]:
df1 = df_train_ins.groupby(['cat_id'])['id'].count().reset_index(name='total_entries')
df2 = df_train_ins.groupby(['cat_id', 'state_id'])['total_sales_all_days'].sum().reset_index()
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5))

sns.barplot(x = 'cat_id', y='total_entries', data=df1, 
            palette='mako', ax=ax1)
sns.barplot(x = 'cat_id', y='total_sales_all_days', hue='state_id', data=df2, 
            palette='magma', ax=ax2)

plt.xticks(rotation=90)
plt.show()
del df1, df2

The above two plots indicate:
- Most items sold belong to the FOODS category, followed by HOUSEHOLD and HOBBIES.
- CA leads in the number of "Total items" sold in either category (FOODS, HOBBIES AND HOUSEHOLD) , while WISCONSIN lags behind TX in each category except FOODS. We would see whether the same difference translates in terms of revenue extracted off these states or not.

In [ ]:
df3 = df_train_ins.groupby(['cat_id', 'store_id'])['total_sales_all_days'].sum().reset_index()
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
fig, ax1 = plt.subplots(figsize=(14,5))

sns.barplot(x = 'store_id', y='total_sales_all_days', hue='cat_id', data=df3, 
            palette='afmhot', ax=ax1)

plt.xticks(rotation=90)
plt.show()
del df3

In [ ]:
df = df_train_ins.groupby(['state_id', 'cat_id'])['id'].count().reset_index(name='num_sales_by_category')
fig = px.bar(df, x="state_id", y="num_sales_by_category", 
             color="cat_id", title="Distribution of Product_ids Count Across Categories & Each Locale")
fig.show()
del df

A couple of points that could be drawn from this observation are:

Most items have been sold in california
Texas and Wisconsin stores have almost total sales i.e. during the same timeframe of 1913 days, same number of items had been sold in both Texas and Wisconsin. Would this observation hold true in the df_sell_prices (revenue dataset) ? Does the observation remain same across different store locations in both Texas and Wisonconsin?

#### Distribution of Items Across Department & Store_ids?

In [ ]:
df = df_train_ins.groupby(['dept_id', 'store_id', 'state_id', 'cat_id'])[df_train_ins.columns[6:]].sum().reset_index()
df = df.sort_values('total_sales_all_days', ascending=False)

In [ ]:
x_dept = df['dept_id']
x_store = df['store_id']

def items_sold_per_days(x_spec,title_text, title):
    
    '''
    returns plotly plots with drop down menus for specified parameter made in dataframe earlier
    
    inputs: x_spec (categorical feature on the x_axis), title_text(title on dropdown), 
            title (title of the plot)
            
    returns: plotly plots of categorical feature (x_axis) with dropdowns on specific 
    number of days        
    '''
    
    cols = ['d_1', 'd_50', 'd_300', 'd_500', 'd_700', 'd_900', 'd_1100', 'd_1500', 'd_1700',
        'total_sales_all_days', 'median_sales_all_days']

    buttons1 = [dict(method = "restyle",
                 args = [{'x': [x_spec, 'undefined'],
                          'y': [df[cols[k]], 'undefined'],
                          'visible':[True, False]}], 
                 label = cols[k])   for k in range(0, len(cols))]

    fig = go.Figure()
    fig.add_trace(go.Bar(x=x_spec, y = df['d_1'], name='Dept.Sales on day2',
                     marker_color='Crimson'))

    fig.update_layout(title_text= title_text,
                  title_x= 0.4, width=750, height=450, 
                  margin=dict(t=100, b=20, l=0, r=0),
                  autosize = False,
                  updatemenus=[dict(active=0,
                                    buttons=buttons1,
                                    x=0.08,
                                    y=1.13,
                                    xanchor='left',
                                    yanchor='top')
                              ]); 

    fig.add_annotation( x=0.00,  y=1.13, showarrow=False, xref='paper', yref='paper', xanchor='left',
                   text="With<br>"+str(title));
    fig.show()

items_sold_per_days(df['store_id'], 'Distribution of Sales Made on Each Store', 'Stores')
items_sold_per_days(df['state_id'], "Distribution of Sales Made In Each State", 'States')

The data_analysis on these points helps make it clear that:
- Regarding the distribution of sales across department ids, most sales have been made across "FOODS_3" category followed by most sales made across household_1 category
- Stores location identity along with embedded state_ids  make it clear that the distribution of sales across Texas & Wisconsin stores are NOT the same , though the total number represented across categories (foods, houshold and hobbies) might have come the same.
- The outperformers in each state of CA, TX and WI are the stores with ids CA_3, TX_2 and WI_3, respectively.

#### Specific item outselling the most?

- Since the df_train_sales contains the information about each specific item and the number of sales made, we can make a few observations regarding the most frequently purchased item too.

- We could plot its behavior across the number of days to get a general gist of its sales pattern across given days. i.e d_1 to d_1913

In [ ]:
df_train_ins.groupby(['id'])['total_sales_all_days'].sum().sort_values(ascending=False)

Once the total number of sales have been grouped against specific item id i.e. 'id' parameter, it is clear that the item "**FOODS_3_090_CA_3_validation**" has clearly sold most units than any other item in the category followed by **"FOODS_3_586_TX_2_validation"**. i.e. the first item belongs to food_3 category and sold in the CA_3 store location. Similarly, the second one belongs to TX_2 store location (i.e. second store in Texas) also belonging to the same category of FOODS_3 which is also consistent with observations made before.

In [ ]:
sns.set_style('whitegrid')

def plot_dailysales(spec_id):
    """
    plots the behavior of dailysales of specific ids i.e. spec_id
    
    input: spec_id
    returns : number of sales plotted across number of days 
    """
    d_cols = [c for c in df_train.columns if 'd_' in c]
    df_train.loc[df_train['id'] == spec_id ].set_index('id')[d_cols]\
                .T\
                .plot(figsize = (13,2.5),
                      title =  str(spec_id)+"_item daily sales", 
                      color = next(color_cycle) )
    plt.legend()
    plt.show()

plot_dailysales('FOODS_3_090_CA_3_evaluation') 
plot_dailysales('FOODS_3_586_TX_2_evaluation')

#### Item ID outselling most in each category?

In [ ]:
df_agg = pd.DataFrame(df_train_ins.groupby(['id', 'cat_id', 'store_id'])['total_sales_all_days'].sum().sort_values(ascending=False))
df_agg = df_agg.reset_index()
df_agg.head(3)

Now that the dataset has been arranged in descending order of total sales, it would be a lot easier to estimate the item_id ('id') outselling others in each category.

In [ ]:
print("The 3 item_ids outselling most in FOODS category are: {}".format(list(df_agg.loc[df_agg['cat_id'] == 'FOODS']['id'][:3])))
print("The 3 item_ids outselling most in HOUSEHOLDS category are: {}".format(list(df_agg.loc[df_agg['cat_id'] == 'HOUSEHOLD']['id'][:3])))
print("The 3 item_ids outselling most in HOBBIES category are: {}".format(list(df_agg.loc[df_agg['cat_id'] == 'HOBBIES']['id'][:3])))

In [ ]:
d_cols = [c for c in df_train.columns if 'd_' in c]
df = pd.DataFrame({"days": list(df_train[df_train['id'] == 'FOODS_3_090_CA_3_evaluation'][d_cols].columns),
                   "sales_data": list(df_train[df_train['id'] == 'FOODS_3_090_CA_3_evaluation'][d_cols].values.flatten())})

del d_cols, df

In [ ]:
#fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(14, 7))

def plot_sample_sales(spec_id, sm_start, sm_end, samples_pick=50):
    """
    plots sample sales data with selection point and ending point specified, along with
    samples_pick point that specifies the samples picked after specified intervals
    
    input: spec_id (item_id or id), sm_start (sample_start), sm_end(sample_end),
    samples_pick (samples picked after how many intervals)
    
    returns: outputs a graph of sample points plotted against daily sales data d_1 to d_1913
    """
    d_cols = [c for c in df_train.columns if 'd_' in c]
    fig, ax1 = plt.subplots(figsize=(13, 2.5))
    
    x1 = list(df_train[df_train['id'] == spec_id][d_cols]\
              .columns)[sm_start:sm_end]
    y1 = list(df_train[df_train['id'] == spec_id][d_cols]\
              .values.flatten())[sm_start:sm_end]
    
    #this conversion for regplot only
    x1 = [x.replace("d_", "") for x in x1]
    x1 = [int(x) for x in x1]
    
    #sns.lineplot(x=x1, y=y1, ax=ax1)
    sns.regplot(x=x1, y=y1, order=10, ax=ax1)
    ax1.set_ylabel("Number of Sales")
    ax1.set_xlabel("Days")

    ax1.set_xticks(x1[::samples_pick])
    ax1.set_xticklabels(x1[::samples_pick], rotation=0)

    fig.tight_layout()
    plt.show()

In [ ]:
plot_sample_sales('FOODS_3_090_CA_3_evaluation', 500, 1300)
plot_sample_sales('FOODS_3_586_TX_2_evaluation', 500, 1300)
plot_sample_sales('FOODS_3_090_CA_1_evaluation', 500, 1300)

Looking at the above graphs, our regression model does a fairly good job of fitting the line on the sales trend observed between the days 500th to 900th, for the item ids 'FOODS_3_090_CA_3_validation' and 'FOODS_3_586_TX_2_validation'. The graph also points out to the similar trends of troughs and crests between the specific days pointing out towards the occurence of special occasions and events that are driving sales. (To do _explain on order of polynomial and take more cases of ids across foods, household categories across different stores)

### Questions on Sales Revenue?

- First of all, we'd be interested in finding out the specific revenue with respect to each product? But since we have not been provided the dataset with ids (product_ids) in df_sell_prices(revenue dataframe) and there is a mismatch in dataset entries between the both datasets (df_train_sales with almost 30000 rows and df_sell_prices with almost 6M entries), therefore, atbest, a rough estimate could be made by merging both datasets.

- Fortunately, Revenue Dataframe (df_sell_prices) has the categorical level data available, so we could make an estimation regarding the items sold in each category to see what is the specific price where most items are getting sold. 


In [ ]:
df_sell_prices.head(3)

In [ ]:
# making a new column category out of the item_id 
df_sell_prices_ins = df_sell_prices.copy()
df_sell_prices_ins['category'] = df_sell_prices_ins['item_id'].str.split("_", expand=True)[0]

#### Distribution of price among categories?

In [ ]:
sns.set_style('whitegrid')
#plt.figure(figsize=(15,5))

def kde_plotting(df, category, bin_size, color, label):
    
    '''
    plots the kde density plot of the continuous features of df specified
    
    inputs: df, category(whether, foods, household or hobbies), bin_size(bin size for histogram)
            color (color of the plot), label (label to the plot)
    returns: kde plots with logarithmic scale taken on x_axis
            
    '''
    fig, ax1 = plt.subplots(figsize=(13, 2.5) )

    sns.distplot(df[df['category'] == category]['sell_price'], 
               axlabel = label ,bins=bin_size, color = color, ax=ax1) 

    fig.tight_layout()
    ax1.set_xscale('log')
    plt.legend()
    plt.show()
    
kde_plotting(df_sell_prices_ins, 'HOBBIES', 150, 'b', 'hobbies')   
kde_plotting(df_sell_prices_ins, 'FOODS', 250, 'g', 'foods') 
kde_plotting(df_sell_prices_ins, 'HOUSEHOLD', 150, 'r', 'household') 

# Insights from these kde plots. 

- The probability distribution plot of the **household** items follows an almost normal distribution with a mean centered around a price of 5 Dollars and most items being sold within the 1 to 10 dollars range. This would indicate that most household items that are getting sold lie within the price bracket of 25 cents to 10 dollars
- **Foods** items prices is a multimodal distribution indicating frequent variation in interest among food items purchased. The values occur both towards the relative higher price bracket as well as lower price bracket indicating that the degree of interest of consumers in food items is not only varied but that the Walmart stores have a catalogue of food items that are peaking consumer's interest across different categories. The price bracket in this case also happens to lie within 2 cents to 10 dollars with only very few items getting sold past that range
- The probability distribution of **hobbies** related items prices indicates a mix of bimodal and multimodal distributions. This indicates that while a few items in specific category were sold more than others (first peak that lies in the area between 0.01 dollar to 1 dollar) there are also items towards a relative higher price bracket that have been also sold quite frequently enought to give it a multimodal distribution with small decreasing peaks indicative of decreasing interest in hobbies related items that are relatively expensive but sill significant enough to generate consumer interest. 

# Removing outliers to observe price distribution? 
#### **Quartile Method**

In [ ]:
def remove_outliers(df):
    
    '''
    removes the outliers in continous features using quartile ranges
    
    inputs: df(df specified with continous features along side categorical features)
    returns: df with removed outliers
    '''
    Q1=df.quantile(0.25)
    Q3=df.quantile(0.75)
    IQR=Q3-Q1
    df_final=df[~((df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR)))]
    
    return df_final

df = df_sell_prices_ins[['category', 'sell_price']]
df = remove_outliers(df)

In [ ]:
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
plt.figure(figsize=(13,3))
sns.boxplot(y=df['category'], x=df['sell_price'])
plt.show()
del df

After removing most of the outliers, it is apparent that for 
- FOODS related items, 75 % of the items sold are those that are less than 4 dollars
- HOBBIES related items, 75 % of the items sold were less than 6 dollars with an mean price centered around 3.25-3.5 dollars
- HOUSEHOLD items, 75 % of the items sold were less than 6.5 dollars. 

It also represents that there are quite a few outliers in our price data. Since we had observed before using the kdeplots, that the distributions of the dataset were mostly skewed, we used the quartile method of removing the outliers. 


# Distribution of sales on weekdays & special occasions?

Our third dataset named, df_calendar, provides valuable information along the timeseries for the dataset of product_id. This dataset also contains information about special occasions, SNAP (Supplementary Nutrition Assistance Programme) in the USA and coupled with the product_id dataset i.e. df_train_sales would be helpful in observing sales along weekdays, specific dates and special occasions.

In [ ]:
df_calendar_ins = df_calendar.copy()
df_calendar_ins.head(3)

In [ ]:
df_calendar_ins.groupby(['event_name_1', 'event_type_1'])['wday'].count()

Looking at the distribution of data in 'event_name_1' and 'event_type_1' it is clear the data here relates to holidays which could reveal important trends when coupled with the information of sales made on the specific event.

It seems a few entries that have not been made to the 'event_name_1' attribute have been made available in a different category

In [ ]:
#Creating and including a new entry of days as well as merging the events_1 and event_2 into
# a single new events_names and types category

df_calendar_ins['days'] = [d.split('-')[2] for d in df_calendar_ins['date']]
df_calendar_ins['events_names'] = pd.concat([df_calendar_ins['event_name_1'], df_calendar_ins['event_name_2']], 
                                        ignore_index=True)
df_calendar_ins['events_types'] = pd.concat([df_calendar_ins['event_type_1'], df_calendar_ins['event_type_2']], 
                                        ignore_index=True)
#calendar_df.drop(['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'], axis=1, inplace=True)

### What are **SNAP_CA, SNAP_TX, SNAP_WI**?

SNAP stands for "Supplementary Nutrition Assistance Program" that is a federal level program aimed at providing food essentials to low-income households. This program is geared towards providing the food essentials and within the current dataset, the catagories of household items and hobbies items do not fall within the requirements of this program.

This program is only geared towards fighting the food hunger in america and only food related items can be purchased under this program.

In [ ]:
df = df_calendar_ins.groupby(['events_types'])['snap_CA'].value_counts().reset_index(name='counts')
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
plt.figure(figsize=(15,5))
sns.barplot(x = 'events_types', y='counts', hue='snap_CA', data=df, palette='bwr')
plt.show()
del df

In [ ]:
df = df_calendar_ins.groupby(['events_names'])['snap_CA'].value_counts().reset_index(name='counts')
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
plt.figure(figsize=(13,3))
sns.barplot(x = 'events_names', y='counts', hue='snap_CA', data=df, 
            order = df.sort_values(['counts'], ascending=False).events_names, 
            palette='OrRd')
plt.xticks(rotation=90)
plt.show()
del df

A look at both of these plots indicates the special occasions when the SNAP programme in CA were availed.  

### Distribution of sales items vs sales revenue? 
Now that we have product_id df as well as revenue_df, we'll merge the dataset now to start exploring trends of item_specific_data and sale_price_specific data 

In [ ]:
#product id df (df_sales_train) vs revenue_df(df_sell_prices)
df_sales_prices = df_train_ins.merge(df_sell_prices_ins, how='inner', left_index=True, right_index=True)  

In [ ]:
df = df_sales_prices.groupby(['cat_id', 'state_id', 'store_id_x'])['sell_price'].sum().reset_index(name='total_revenue')
df = df.sort_values(by='total_revenue', ascending=False)
sns.set_style('whitegrid')
sns.axes_style(style='ticks')
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(14,5))

sns.barplot(x = 'state_id', y='total_revenue', data=df, 
            palette='coolwarm', ax=ax1)
sns.barplot(x = 'cat_id', y='total_revenue', hue='state_id', data=df, 
            palette='plasma', ax=ax2)

plt.xticks(rotation=90)
plt.show()
del df, df_sales_prices

Despite the fact that sales prices data contains almost 6M entries, in the present case, we are only considering common entries between revenue df and product_ids df. A few significant insights have come forward, i.e. 

- Although we saw that California consistently was the one state where the unique product_id most sales were made, the most revenue collected came from the Wisconsin State stores.
- Similarly, within the distribution of categories, WI and TX contrinute more sales revenue than the CA stores. 
- Wisconsin leads the revenue in FOODS and HOBBIES, while Texas leads the revenue in HOUSEHOLD.
- CA tends to contribute the smallest revenue out of all three states, despite having the most sales of items in its stores locations.

In [ ]:
gc.collect()

<a id="four"></a>
## 4. Feature Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Feature engineering ⚡ |
| :--------------------------- |
| In this section, we cleaned the dataset, and engineered new features based on insights gained in the EDA section. |

---

In [ ]:
%%time
for col in df_calendar.columns:
    if df_calendar[col].isnull().sum()>0:
        df_calendar[col] = df_calendar[col].cat.add_categories('no_event').fillna('no_event')
        print(df_calendar[col].dtype)

In [ ]:
%%time
#Adding feature 'is_weekend' which tells about that day is weekend or not
f=lambda x: 1 if x<=2 else 0
df_calendar['weekend']=df_calendar['wday'].map(f) 
df_calendar['weekend']=df_calendar['weekend'].astype(np.int8)

In [ ]:
%%time
#Adding feature 'month_day' which tells day of the month
dates = df_calendar["date"].tolist()
dates=[i.split("-")[2] for i in dates]
df_calendar["month_day"]=dates
df_calendar['month_day']=df_calendar['month_day'].astype(np.int8)
"""
for col in df_calendar.columns:
    if df_calendar[col].dtype == 'int16':
        df_calendar[col] = df_calendar[col].astype(np.int8)
"""
df_calendar.info()

In [ ]:
%%time
#Adding feature 'month_week_number' which tells which week of the month
df_calendar['month_week_number'] = (df_calendar['month_day']-1) // 7 + 1 
df_calendar['month_week_number'] = df_calendar['month_week_number'].astype(np.int8)

df_calendar['month_week_number'].unique()

In [ ]:
%%time
#Adding feature 'events_per_day' which tells us number of events on particular day
f=lambda x: 0 if x=='no_event' else 1
df_calendar['events_per_day']=df_calendar['event_type_1'].map(f) 
index=df_calendar.index 
indices=index[df_calendar['event_type_2']!='no_event'].tolist()
for i in indices:
    df_calendar['events_per_day'][i]+=1
df_calendar['events_per_day']=df_calendar['events_per_day'].astype(np.int8)

<a id="4.1"></a>
### 4.1 `Melting` of Dataframe

---

In [ ]:
%%time
# This is the cell that requires more memory to run

# creating a single dataframe

final_df = pd.melt(
                    df_train,
                    id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                    var_name="d",
                    value_name="sales",
                    )
del df_train
gc.collect()

In [ ]:
%%time
final_df#.info()

<a id="4.2"></a>
### 4.2 `Merging` of Dataframe

---

In [ ]:
%%time
final_df = pd.merge(
                    final_df,
                    df_calendar,
                    on="d",
                    how="left",
                    )
del df_calendar
gc.collect()

In [ ]:
final_df.head()

In [ ]:
%%time
final_df = pd.merge(
                    final_df,
                    df_sell_prices,
                    on=["item_id", "store_id", "wm_yr_wk"],
                    how="left",
                    )
del df_sell_prices
gc.collect()

In [ ]:
final_df.head()

In [ ]:
%%time
final_df = reduce_mem_usage(final_df)

In [ ]:
%%time
final_df['date'] = pd.to_datetime(final_df['date'])

In [ ]:
%%time
final_df.info()

In [ ]:
%%time
final_df.isnull().sum()

In [ ]:
%%time
final_df = reduce(final_df)
gc.collect()

In [ ]:
%%time
final_df.head()

In [ ]:
%%time
final_df['sell_price'] = final_df['sell_price'].fillna(final_df.groupby('id')['sell_price'].transform('mean'))

In [ ]:
%%time
final_df['sell_price'].isnull().sum()

In [ ]:
%%time
final_df = reduce(final_df)
gc.collect()

<a id="4.3"></a>
### 4.3 Engineering `lag` features

---

In [ ]:
%%time
lags=[28,35,42,49,56,63,70]
for i in tqdm(lags):
    final_df['lag_'+str(i)] = final_df.groupby(['id'])['sales'].shift(i).astype('float16').fillna(0)

In [ ]:
%%time
final_df.isnull().sum()

<a id="4.4"></a>
### 4.4 Engineering `rolling` mean features

---

In [ ]:
%%time
#Rolling-Mean
window=[7,14,28,35,42]
for i in tqdm(window):
    final_df['rolling_median_'+str(i)]=final_df.groupby(['id'])['sales'].transform(lambda s: s.rolling(i,center=False).median()).astype('float16').fillna(0)

In [ ]:
%%time
final_df.info()

<a id="4.5"></a>
### 4.5 Dropping Redundant Features

---

In [ ]:
%%time
final_df.drop(['weekday'], axis=1, inplace=True)
final_df.head()

<a id="4.6"></a>
### 4.6 Label Encoding

---

In [ ]:
%%time
final_df.dtypes

In [ ]:
%%time
final_df.rename(columns = {'d': 'day'}, inplace = True)
final_df['day'] = final_df['day'].str.split('_', expand=True).iloc[: , 1:].astype('int16')
final_df.head()

In [ ]:
%%time
le=LabelEncoder()

categories = [col for col in final_df.columns if str(final_df[col].dtypes) == 'category'][6:]
for column_name in tqdm(categories):
    final_df[column_name] = le.fit_transform(final_df[column_name]).astype('int16')

In [ ]:
%%time
final_df

In [ ]:
%%time
final_df = reduce_mem_usage(final_df)

In [ ]:
%%time
final_df = reduce(final_df)

<a id="4.7"></a>
### 4.7 Saving Engineered Features

---

In [ ]:
%%time
#final_df.to_pickle("../sales_data/final_df.pkl")

df_path = "../sales_data/processed_data/final_df.pkl"
with open(df_path,'wb') as file:
    pickle.dump(final_df, file)

<a id="4.8"></a>
### 4.8 Releasing Memory

---

In [ ]:
del final_df, categories
gc.collect()

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, we created 3 Machine Learning Models that are able to accurately forecast future sales. |

---

<a id="5.1"></a>
### 5.1 FacebookProphet

---

<img src="resources/m5_levels_new.png">

Explain the parameters of the model; yhat, yhat_lower, yhat_upper etc.

In [ ]:
%%time
calendar = load_csv_optimized('~/sales_data/raw_data/calendar.csv')
train = load_csv_optimized("~/sales_data/raw_data/sales_train_evaluation.csv")

In [ ]:
def rmse(pred,value):
    return np.sqrt(((pred-value)**2).mean())

<a id="5.1.1"></a>
### 5.1.1 Top_Level

---

In [ ]:
%%time
def facebook_preprocess(df_calendar, df_train):
    series_cols = df_train.columns[df_train.columns.str.contains("d_")].values
    timeseries = df_train[series_cols].sum().values
    train_timeseries = timeseries[0:-28]
    eval_timeseries = timeseries[-28::]
    days = np.arange(1, len(series_cols)+1)
    dates = calendar.iloc[0:len(timeseries)].date.values
    df = pd.DataFrame(dates, columns=["ds"])
    df.loc[:, "y"] = timeseries
    train_df = df.iloc[0:-28]
    eval_df = df.iloc[-28::]
    
    return train_df, eval_df

def facebook_predicting(train):
    uncertainty_interval_width = 0.25
    m = Prophet(interval_width=uncertainty_interval_width, daily_seasonality=True)
    m.fit(train)
    return m

def facebook_future(m, period):
    future = m.make_future_dataframe(periods=period)
    forecast = m.predict(future)
    col_int = ['ds', 'yhat', 'yhat_lower', 'yhat_upper']
    forecast[col_int].head()
    return forecast[col_int]

In [ ]:
%%time
train_, eval_ = facebook_preprocess(calendar, train)
prophet_full = facebook_predicting(train_)
pred = facebook_future(prophet_full, 28)
pred

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_full_data.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_full, file)

In [ ]:
def facebook_plot(df_calendar, df_train):
    series_cols = df_train.columns[df_train.columns.str.contains("d_")].values
    timeseries = df_train[series_cols].sum().values
    train_timeseries = timeseries[0:-28]
    eval_timeseries = timeseries[-28::]
    days = np.arange(1, len(series_cols)+1)
    plt.figure(figsize=(20,5))
    plt.plot(days[0:-28], train_timeseries, label="Train")
    plt.plot(days[-28::], eval_timeseries, label="Validation")
    plt.title("Top-Level-1: Summed product sales of all stores and states");
    plt.legend()
    plt.xlabel("Day")
    plt.ylabel("Unit sales");
    
facebook_plot(calendar, train)

In [ ]:
%%time
forecast = facebook_future(prophet_full, 28)
plt.plot(forecast.iloc[-28::].yhat.values, 'o', label="predicted yhat")
plt.plot(eval_.y.values, 'o-', label="target")
plt.legend();

<a id="5.1.2"></a>
### 5.1.2 State_Level

---

In [ ]:
%%time
for col in train['state_id'].unique():
    #print(col)
    if str(col) == "CA":
        df_CA = train[train['state_id']==str(col)]
        #print(df_CA)
    elif str(col) == "TX":
        df_TX = train[train['state_id']==str(col)]
    else:
        df_WI = train[train['state_id']==str(col)]
    #print(col)

In [ ]:
%%time
train_ca, eval_ca = facebook_preprocess(calendar, df_CA)
prophet_ca = facebook_predicting(train_ca)
facebook_future(prophet_ca, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_state_ca.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca, file)

In [ ]:
%%time
train_tx, eval_tx = facebook_preprocess(calendar, df_TX)
prophet_tx = facebook_predicting(train_tx)
facebook_future(prophet_tx, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_state_tx.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_tx, file)

In [ ]:
%%time
train_wi, eval_wi = facebook_preprocess(calendar, df_WI)
prophet_wi = facebook_predicting(train_wi)
facebook_future(prophet_wi, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_state_wi.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_wi, file)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_state_wi.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_wi, file)

In [ ]:
%%time

del df_CA, df_TX, df_WI, train_ca, eval_ca, train_tx, eval_tx, train_wi, eval_wi, prophet_ca, prophet_tx, prophet_wi

<a id="5.1.3"></a>
### 5.1.3 Store_Level

---

In [ ]:
%%time
for col in train['store_id'].unique():
    #print(col)
    if str(col) == "CA_1":
        df_CA_1 = train[train['store_id']==str(col)]
    elif str(col) == "CA_2":
        df_CA_2 = train[train['store_id']==str(col)]
    elif str(col) == "CA_3":
        df_CA_3 = train[train['store_id']==str(col)]
    elif str(col) == "CA_4":
        df_CA_4 = train[train['store_id']==str(col)]
    elif str(col) == "TX_1":
        df_TX_1 = train[train['store_id']==str(col)]
    elif str(col) == "TX_2":
        df_TX_2 = train[train['store_id']==str(col)]
    elif str(col) == "TX_3":
        df_TX_3 = train[train['store_id']==str(col)]
    elif str(col) == "WI_1":
        df_WI_1 = train[train['store_id']==str(col)]
    elif str(col) == "WI_2":
        df_WI_2 = train[train['store_id']==str(col)]
    else:
        df_WI_3 = train[train['store_id']==str(col)]
    #print(col)

In [ ]:
%%time
train_ca_1, eval_ca_1 = facebook_preprocess(calendar, df_CA_1)
prophet_ca_1 = facebook_predicting(train_ca_1)
facebook_future(prophet_ca_1, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_ca_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_1, file)

In [ ]:
%%time
train_ca_2, eval_ca_2 = facebook_preprocess(calendar, df_CA_2)
prophet_ca_2 = facebook_predicting(train_ca_2)
facebook_future(prophet_ca_2, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_ca_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_2, file)

In [ ]:
%%time
train_ca_3, eval_ca_3 = facebook_preprocess(calendar, df_CA_3)
prophet_ca_3 = facebook_predicting(train_ca_3)
facebook_future(prophet_ca_3, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_ca_3.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_3, file)

In [ ]:
%%time
train_ca_4, eval_ca_4 = facebook_preprocess(calendar, df_CA_4)
prophet_ca_4 = facebook_predicting(train_ca_4)
facebook_future(prophet_ca_4, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_ca_4.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_4, file)

In [ ]:
%%time
train_tx_1, eval_tx_1 = facebook_preprocess(calendar, df_TX_1)
prophet_tx_1 = facebook_predicting(train_tx_1)
facebook_future(prophet_tx_1, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_tx_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_tx_1, file)

In [ ]:
%%time
train_tx_2, eval_tx_2 = facebook_preprocess(calendar, df_TX_2)
prophet_tx_2 = facebook_predicting(train_tx_2)
facebook_future(prophet_tx_2, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_tx_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_tx_2, file)

In [ ]:
%%time
train_tx_3, eval_tx_3 = facebook_preprocess(calendar, df_TX_3)
prophet_tx_3 = facebook_predicting(train_tx_3)
facebook_future(prophet_tx_3, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_tx_3.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_tx_3, file)

In [ ]:
%%time
train_wi_1, eval_wi_1 = facebook_preprocess(calendar, df_WI_1)
prophet_wi_1 = facebook_predicting(train_wi_1)
facebook_future(prophet_wi_1, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_wi_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_wi_1, file)

In [ ]:
%%time
train_wi_2, eval_wi_2 = facebook_preprocess(calendar, df_WI_2)
prophet_wi_2 = facebook_predicting(train_wi_2)
facebook_future(prophet_wi_2, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_wi_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_3, file)

In [ ]:
%%time
train_wi_3, eval_wi_3 = facebook_preprocess(calendar, df_WI_3)
prophet_wi_3 = facebook_predicting(train_wi_3)
facebook_future(prophet_wi_3, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_store_wi_3.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_ca_3, file)

<a id="5.1.4"></a>
### 5.1.4 Category_Level

---

In [ ]:
%%time
df_cat_hobbies = train[train['cat_id'] == "HOBBIES"]
train_hob, eval_hob = facebook_preprocess(calendar, df_cat_hobbies)
prophet_cat_hob = facebook_predicting(train_hob)
facebook_future(prophet_cat_hob, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_cat_hobbies.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_hob, file)
    
del df_cat_hobbies, train_hob, eval_hob, prophet_cat_hob, df_path

In [ ]:
%%time
df_cat_household = train[train['cat_id'] == "HOUSEHOLD"]
train_house, eval_house = facebook_preprocess(calendar, df_cat_household)
prophet_cat_house = facebook_predicting(train_house)
facebook_future(prophet_cat_house, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_cat_household.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_house, file)
    
del df_cat_household, train_house, eval_house, prophet_cat_house, df_path

In [ ]:
%%time
df_cat_food = train[train['cat_id'] == "FOODS"]
train_food, eval_food = facebook_preprocess(calendar, df_cat_food)
prophet_cat_food = facebook_predicting(train_food)
facebook_future(prophet_cat_food, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_cat_food.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_food, file)
    
del df_cat_food, train_food, eval_food, prophet_cat_food, df_path

<a id="5.1.5"></a>
### 5.1.5 Sub_Category_Level

---

In [ ]:
%%time
df_cat_hobbies = train[train['dept_id'] == "HOBBIES_1"]
train_hob, eval_hob = facebook_preprocess(calendar, df_cat_hobbies)
prophet_cat_hob = facebook_predicting(train_hob)
facebook_future(prophet_cat_hob, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_hobbies_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_hob, file)
    
del df_cat_hobbies, train_hob, eval_hob, prophet_cat_hob, df_path

In [ ]:
%%time
df_cat_hobbies = train[train['dept_id'] == "HOBBIES_2"]
train_hob, eval_hob = facebook_preprocess(calendar, df_cat_hobbies)
prophet_cat_hob = facebook_predicting(train_hob)
facebook_future(prophet_wi_2, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_hobbies_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_hob, file)
    
del df_cat_hobbies, train_hob, eval_hob, prophet_cat_hob, df_path

In [ ]:
%%time
df_cat_household = train[train['dept_id'] == "HOUSEHOLD_1"]
train_house, eval_house = facebook_preprocess(calendar, df_cat_household)
prophet_cat_house = facebook_predicting(train_house)
facebook_future(prophet_cat_house, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_house_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_house, file)
    
del df_cat_household, train_house, eval_house, prophet_cat_house, df_path

In [ ]:
%%time
df_cat_household = train[train['dept_id'] == "HOUSEHOLD_2"]
train_house, eval_house = facebook_preprocess(calendar, df_cat_household)
prophet_cat_house = facebook_predicting(train_house)
facebook_future(prophet_cat_house, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_house_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_house, file)
    
del prophet_cat_house, train_house, eval_house, df_path

In [ ]:
%%time
df_cat_food = train[train['dept_id'] == "FOODS_1"]
train_food, eval_food = facebook_preprocess(calendar, df_cat_food)
prophet_cat_food = facebook_predicting(train_food)
facebook_future(prophet_cat_food, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_food_1.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_food, file)
    
del df_cat_food, train_food, eval_food, df_path

In [ ]:
%%time
df_cat_food = train[train['dept_id'] == "FOODS_2"]
train_food, eval_food = facebook_preprocess(calendar, df_cat_food)
prophet_cat_food = facebook_predicting(train_food)
facebook_future(prophet_cat_food, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_food_2.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_food, file)
    
del df_cat_food, prophet_cat_food, train_food, eval_food, df_path

In [ ]:
%%time
df_cat_food = train[train['dept_id'] == "FOODS_3"]
train_food, eval_food = facebook_preprocess(calendar, df_cat_food)
prophet_cat_food = facebook_predicting(train_food)
facebook_future(prophet_cat_food, 28)

In [ ]:
%%time
df_path = "../sales_data/pickled_models/facebook_models/prophet_sub_cat_food_3.pkl"
with open(df_path,'wb') as file:
    pickle.dump(prophet_cat_food, file)
    
del df_cat_food, train_food, eval_food, prophet_cat_food, df_path

## Putting all the bits together...

In [ ]:
%%time
AGGREGATION_LEVELS = [
    [],
    ['state_id'],
    ['store_id'],
    ['cat_id'],
    ['dept_id'],
    ['state_id', 'cat_id'],
    ['state_id', 'dept_id'],
    ['store_id', 'cat_id'],
    ['store_id', 'dept_id'],
    ['item_id'],
    ['state_id', 'item_id'],
    ['item_id', 'store_id']
]
INTERVALS = [0.99, 0.95, 0.75, 0.5]

In [ ]:
%%time
def read_sales(filename):
    calendar = pd.read_csv('../sales_data/raw_data/calendar.csv', parse_dates=['date'])
    sales = pd.read_csv(filename)
    agg_sales = []
    for level in AGGREGATION_LEVELS:
        if len(level) == 0:
            agg = pd.DataFrame(sales.sum(numeric_only=True)).T
            agg['id'] = 'Total_X'
        elif len(level) == 1:
            agg = sales.groupby(level).sum(numeric_only=True).reset_index()
            agg['id'] = agg[level[0]] + '_X'
            agg.drop(level, axis=1, inplace=True)
        else:
            agg = sales.groupby(level).sum(numeric_only=True).reset_index()
            agg['id'] = agg[level[0]] + '_' + agg[level[1]]
            agg.drop(level, axis=1, inplace=True)
        agg_sales.append(agg)
    sales = pd.concat(agg_sales)
    sales.set_index('id', inplace=True)
    sales.columns = calendar.date[:len(sales.columns)]
    return sales

In [ ]:
%%time
def fit_model(params):
    data, prefix, suffix = params
    data = data.T.reset_index()
    data.columns = ['ds', 'y']
    quantiles = []
    for interval in INTERVALS:
        model = Prophet(interval_width=interval,
                       daily_seasonality=True)
        model.fit(data)
        future = model.make_future_dataframe(periods=28)
        forecast = model.predict(future)
        quantile = forecast[['ds', 'yhat_lower', 'yhat_upper']].tail(28).copy()
        lower = (1 - interval) / 2
        upper = 1 - lower
        quantile.columns = ['date', f'{prefix}_{lower:.3f}_{suffix}', f'{prefix}_{upper:.3f}_{suffix}']
        quantile = quantile.set_index('date').T
        quantile.index.name = 'id'
        quantiles.append(quantile)
    median = forecast[['ds', 'yhat']].tail(28).copy()
    median.columns = ['date', f'{prefix}_0.500_{suffix}']
    median = median.set_index('date').T
    median.index.name = 'id'
    quantiles.append(median)
    quantiles = pd.concat(quantiles)
    return quantiles

In [ ]:
%%time
def forecast(sales, suffix='validation'):
    sales_list = [(row, row.name, suffix) for _, row in sales.head(8).iterrows()]  # for kaggle env
    # sales_list = [(row, row.name, suffix) for _, row in sales.iterrows()]
    pool = Pool(4)
    result = pool.map(fit_model, sales_list)
    return pd.concat(result)

In [ ]:
%%time
sales_valid = read_sales('../sales_data/raw_data/sales_train_validation.csv')

In [ ]:
%%time
sales_eval = read_sales('../sales_data/raw_data/sales_train_evaluation.csv')

In [ ]:
%%time
def forecast_one(index=0):
    data = sales_valid.iloc[index]
    params = data, data.name, 'plot'
    forecast_valid = fit_model(params)
    data = sales_eval.iloc[index]
    params = data, data.name, 'plot'
    forecast_eval = fit_model(params)
    data = pd.concat([forecast_valid, forecast_eval], axis=1)
    data = pd.concat([sales_eval.iloc[index:index+1, -28:], data])
    return data

In [ ]:
%%time
result = forecast_one()
result

In [ ]:
%%time
result.T.plot(figsize=(16,9))

In [ ]:
del sales_eval, sales_valid, result, AGGREGATION_LEVELS, INTERVALS

<a id="5.2"></a>
### 5.2 LightGBModel

---

In [ ]:
%%time
import pickle
df_path = "../sales_data/processed_data/final_df.pkl"
with open(df_path,'rb') as file:
    df = pickle.load(file)

df

In [ ]:
%%time
def rmse(pred,value):
    return np.sqrt(((pred-value)**2).mean())

In [ ]:
%%time
df=df[df['day']>1000]#.drop('date', axis=1, inplace=True) #to save memory, we use days greater 1,000

In [ ]:
df.drop('date', axis=1, inplace=True)

In [ ]:
%%time
l=[]
for i in range(1886,1914):
  l.append(i)

#https://www.kite.com/python/answers/how-to-select-rows-by-multiple-label-conditions-with-pandas-loc-in-python
x_train=df.loc[df['day']<=1885]
x_valid=df.loc[df['day'].isin(l)]
x_test=df.loc[df['day']>=1914]

y_train=x_train['sales']
y_valid=x_valid['sales']
y_test=x_test['sales']

x_train=x_train.drop(['sales'],axis=1)
x_valid=x_valid.drop(['sales'],axis=1)
x_test=x_test.drop(['sales'],axis=1)

print("x_train {}".format(x_train.shape),"  y_train {}".format(y_train.shape))
print("\nx_valid {}".format(x_valid.shape),"  y_valid {}".format(y_valid.shape))
print("\nx_test {}".format(x_test.shape),"  y_test {}".format(y_test.shape))

In [ ]:
%%time
for i in tqdm(range(15)):
    learning_rate=np.round(np.random.uniform(0.001,0.05),4)
    max_depth=np.random.randint(5,100)
    num_leaves=np.random.randint(20,100)
    lgb=LGBMRegressor(learning_rate=learning_rate,max_depth=max_depth,num_leaves=num_leaves,n_jobs=-1,n_estimators=100)
    lgb.fit(x_train,y_train)
    y_pred=lgb.predict(x_valid)
    print("\n\nlearning_rate: {}".format(learning_rate),"  max_depth: {}".format(max_depth),
          " num_leaves: {}".format(num_leaves)," Rmse: {}".format(rmse(y_pred,y_valid)))
    print("-"*100)

In [ ]:
%%time
learning_rate=0.034 
max_depth=66 
num_leaves=224 
lgb=LGBMRegressor(learning_rate=learning_rate,max_depth=max_depth,num_leaves=num_leaves,n_jobs=-1,n_estimators=100)
lgb.fit(x_train,y_train)
y_pred=lgb.predict(x_valid)
print("learning_rate: {}".format(learning_rate),"  max_depth: {}".format(max_depth),"  num_leaves: {}".format(num_leaves),"  Rmse: {}".format(rmse(y_pred,y_valid)))

In [ ]:
%%time
features=x_train.columns
imp=lgb.feature_importances_
indices=(np.argsort(imp))[5:]
plt.figure(figsize=(8,10))
plt.title('Feature Importance',fontsize=14)
plt.barh(range(len(indices)),imp[indices],color='r')
plt.yticks(range(len(indices)),[features[i] for i in indices])
plt.xlabel('Relative Importance',fontsize=14)
plt.show()

In [ ]:
gc.collect()

<a id="5.3"></a>
### 5.3 TensorFlow LSTM

---

We made use of the top level timeseries for an example.


In [ ]:
%%time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
%%time
class MyLSTM(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, batch_size, num_layers=1, output_dim=1):
        super().__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size=self.input_dim,
                            hidden_size=self.hidden_dim,
                            num_layers=self.num_layers,
                            dropout = 0.25)
        self.linear = nn.Linear(self.hidden_dim, output_dim)
        
    def init_hidden(self):
        self.h_zero = torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device)
        self.c_zero = torch.zeros(self.num_layers, self.batch_size, self.hidden_dim).to(device)
    
    def forward(self, x):
        lstm_output, (h_n, c_n) = self.lstm(x.view(len(x), self.batch_size, -1),
                                           (self.h_zero, self.c_zero))
        last_time_step = lstm_output.view(self.batch_size, len(x), self.hidden_dim)[-1]
        pred = self.linear(last_time_step)
        return pred
    

def train_model(model, data_dict, lr=1e-4, num_epochs=500):
    
    loss_fun = torch.nn.MSELoss(reduction="mean")
    optimiser = torch.optim.Adam(model.parameters(), lr=lr)
    
    train_losses = np.zeros(num_epochs)
    phases = ["train", "eval"]
    losses_dict = {"train": [], "eval": []}
    predictions_dict = {"train": [], "eval": [] }
    
    for n in range(num_epochs):
        
        for phase in phases:
            
            x = data_dict[phase]["input"].to(device, dtype=torch.float)
            y = data_dict[phase]["target"].to(device, dtype=torch.float)
            
            if phase == "train":
                model.train()
            else:
                model.eval()
        
            optimiser.zero_grad()
            
            model.init_hidden()
            y_pred = model(x)
            
            if n == (num_epochs-1):
                predictions_dict[phase] = y_pred.float().cpu().detach().numpy()
            
            loss = loss_fun(y_pred.float(), y)
            losses_dict[phase].append(loss.item())
            
            if n % 50 == 0:
                print("{} loss: {}".format(phase, loss.item()))
            
            if phase == 'train':
                loss.backward()
                optimiser.step()
        
    return losses_dict, predictions_dict

def create_sequences(timeseries, seq_len):
    inputs = []
    targets = []
    
    max_steps = len(timeseries) - (seq_len+1)
    
    for t in range(max_steps):
        x = timeseries[t:(t+seq_len)]
        y = timeseries[t+seq_len]
        inputs.append(x)
        targets.append(y)
    
    return np.array(inputs), np.array(targets)

In [ ]:
%%time
series_cols = train.columns[train.columns.str.contains("d_")].values
timeseries = train[series_cols].sum().values
train_timeseries = timeseries[0:-28]
eval_timeseries = timeseries[-28::]
days = np.arange(1, len(series_cols)+1)
dates = calendar.iloc[0:len(timeseries)].date.values

To prepare the data for modelling, we will have to remove the timeseries and scale the values.

In [ ]:
%%time
diff_series = np.diff(timeseries)
train_size = np.int(0.7 * len(diff_series))
train_diff_series = diff_series[0:train_size]
eval_diff_series = diff_series[train_size::]
scaler = MinMaxScaler(feature_range=(-1,1))
scaled_train = scaler.fit_transform(train_diff_series.reshape(-1, 1))
scaled_eval = scaler.transform(eval_diff_series.reshape(-1,1))

In [ ]:
%%time
fig, ax = plt.subplots(1,2,figsize=(20,5))
ax[0].plot(scaled_train, '-o', c="b")
ax[1].plot(scaled_eval, '-o', c="g")
ax[0].set_title("Single preprocessed top timeseries in train")
ax[1].set_title("Single preprocessed top timeseries in eval");
ax[0].set_xlabel("Days in dataset")
ax[1].set_xlabel("Days in dataset")
ax[0].set_ylabel("$\Delta y$ scaled")
ax[1].set_ylabel("$\Delta y$ scaled");


Now, let's prepare the model for fitting.

In [ ]:
%%time
seq_len = 400
input_dim = 1
hidden_dim = 128
num_epochs = 600
lr=0.0005


x_train, y_train = create_sequences(scaled_train, seq_len)
x_eval, y_eval = create_sequences(scaled_eval, seq_len)

x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()
x_eval = torch.from_numpy(x_eval).float()
y_eval = torch.from_numpy(y_eval).float()

data_dict = {"train": {"input": x_train, "target": y_train},
             "eval": {"input": x_eval, "target": y_eval}}

In [ ]:
%%time
model = MyLSTM(input_dim=input_dim,
               hidden_dim=hidden_dim,
               batch_size=seq_len)
model = model.to(device)

In [ ]:
%%time
run_training = True
if run_training:
    losses_dict, predictions_dict = train_model(model, data_dict, num_epochs=num_epochs, lr=lr)
    print("")

In [ ]:
%%time
if run_training:
    
    fig, ax = plt.subplots(3,1,figsize=(20,20))
    ax[0].plot(losses_dict["train"], '.-', label="train", c="red")
    ax[0].set_xlabel("Epochs")
    ax[0].set_ylabel("MSE")
    ax[0].plot(losses_dict["eval"], '.-', label="eval", c="blue");
    ax[0].legend();

    ax[1].plot(predictions_dict["train"], '-o', c="red")
    ax[1].plot(y_train, '-o', c="green")
    ax[1].set_title("Fitted and true values of y in train");
    ax[1].set_ylabel("Unit sales y");
    ax[1].set_xlabel("Number of days in train");

    ax[2].plot(predictions_dict["eval"], '-o', c="red")
    ax[2].plot(y_eval, '-o', c="green")
    ax[2].set_title("Predicted and true values of y in eval");
    ax[2].set_xlabel("Number of days in eval");
    ax[2].set_ylabel("Unit sales y");

In [ ]:
%%time
from statsmodels.graphics.tsaplots import plot_acf

if run_training:
    
    train_residuals = y_train-predictions_dict["train"]
    eval_residuals = y_eval-predictions_dict["eval"]
    
    fig, ax = plt.subplots(2,2,figsize=(20,10))
    sns.distplot(train_residuals, ax=ax[0,0], color="red")
    sns.distplot(eval_residuals, ax=ax[0,1], color="green")
    ax[0,0].set_title("Train residuals")
    ax[0,1].set_title("Eval residuals")
    ax[0,0].set_xlabel("$y_{true} - y_{pred}$")
    ax[0,1].set_xlabel("$y_{true} - y_{pred}$")
    ax[0,0].set_ylabel("density")
    ax[0,1].set_ylabel("density")
    
    plot_acf(train_residuals, ax=ax[1,0])
    plot_acf(eval_residuals, ax=ax[1,1])

In [ ]:
%%time
sampled_residuals = np.random.choice(train_residuals[:, 0], size=len(y_train), replace=True)
sampled_residuals = sampled_residuals.reshape(-1,1)
new_response = predictions_dict["train"] + sampled_residuals

In [ ]:
%%time
fig, ax = plt.subplots(2,2,figsize=(20,10))
ax[0,0].plot(predictions_dict["train"][0:200], 'o-', color="purple")
ax[0,0].set_title("Original fitted values $y_{pred}$ in ")
ax[0,0].set_xlabel("200 example days")
ax[0,0].set_ylim(-0.4, 0.4)
ax[0,0].set_ylabel("$y_{fitted}$")

ax[0,1].plot(new_response[0:200,0], 'o-', color="orange")
ax[0,1].set_title("Response values $y^{*}$ using sampled residuals");
ax[0,1].set_xlabel("200 example days")
ax[0,1].set_ylabel("$y^{*}$");
ax[0,1].set_ylim(-0.4, 0.4)

ax[1,0].plot(sampled_residuals[0:200], 'o-', color="cornflowerblue")
ax[1,0].set_title("Sampled residuals")
ax[1,0].set_xlabel("200 example days")
ax[1,0].set_ylabel("$\epsilon$")

ax[1,1].plot(y_train[0:200], 'o-', color="firebrick")
ax[1,1].set_title("True values $y_{train}$")
ax[1,1].set_xlabel("200 example days")
ax[1,1].set_ylabel("$y_{train}$");

In [ ]:
%%time
responses = []
for n in range(100):
    # sample residuals using the historical residuals found in train
    sampled_residuals = np.random.choice(train_residuals[:, 0], size=len(y_eval), replace=True)
    sampled_residuals = sampled_residuals.reshape(-1,1)
    # create a synthetic future timeseries of eval by adding sampled residuals
    new_response = predictions_dict["eval"] + sampled_residuals
    # reverse the scaling
    new_response = scaler.inverse_transform(new_response)
    # concat the first value of the evaluation series and the response series
    new_response = np.hstack((timeseries[train_size], new_response[:,0]))
    # reverse the differnciation (trend removal) using cumsum
    new_response = np.cumsum(new_response)
    # save the future timeseries
    responses.append(new_response)
    
responses = np.array(responses)
responses.shape

In [ ]:
%%time
y_eval.shape

In [ ]:
%%time
median_series = np.median(responses, axis=0)
eval_series = scaler.inverse_transform(y_eval)
eval_series = np.cumsum(np.hstack((timeseries[train_size-1], eval_series[:,0])))
low_q = 0.25
up_q = 0.75

In [ ]:
%%time
plt.figure(figsize=(20,5))
plt.plot(np.arange(0, len(median_series)), median_series, 'o-', label="median predicted series")
plt.plot(eval_series, '.-', color="cornflowerblue", label="true eval series")
lower = np.quantile(responses, low_q, axis=0)
upper = np.quantile(responses, up_q, axis=0)
plt.fill_between(np.arange(0, len(median_series)), lower, upper, alpha=0.5)
plt.title("Prediction interval {}% of eval timeseries".format((up_q-low_q)*100));
plt.xlabel("Days in eval")
plt.ylabel("Unit sales");
plt.legend();

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section, we compared the relative performance of the 3 ML models on a holdout dataset and made comments on what model is the best and why. |

---

In [ ]:
import tabletext

s = [['No.','Model Name','RMSE'],
        ['1','Facebook Prophet',3.2802],
        ['2','LightGBM Regressor',1.8299],
        ['3','LSTM',1.8667]
      ]
print(tabletext.to_text(s))

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, we discussed how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

#### LGBM - Light Gradient Boosting Machine
As seen from the above models, the `LGBMRegressor model`, from the `LGBM` module performs best among other models.

It is a gradient boosting framework that makes use of tree based learning algorithms that is considered to be a very powerful algorithm when it comes to computation. It is considered to be a fast processing algorithm.


While other algorithms trees grow horizontally, LightGBM algorithm grows vertically meaning it grows leaf-wise and other algorithms grow level-wise. LightGBM chooses the leaf with large loss to grow.

It uses two novel techniques: **Gradient-based One Side Sampling (GOSS)** and **Exclusive Feature Bundling (EFB)** which fulfills the limitations of histogram-based algorithm that is primarily used in all GBDT (Gradient Boosting Decision Tree) frameworks. The two techniques of **GOSS** and **EFB** comprise together to make the model work efficiently and provide it a cutting edge over other GBDT frameworks <a href=#ref2>((LightGBM (Light Gradient Boosting Machine), 2021)</a>.



The LightGBM model is considered to be a really fast algorithm and (arguably) the most used algorithm in machine learning when it comes to getting fast and high accuracy results <a href=#ref3>(What Is LightGBM Algorithm, How to Use It?, 2020)</a>. This is probably because of its `light` computation power and giving results faster. It also takes less memory to run and is able to deal with large amounts of data.

<a id="eight"></a>
## 8. References
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Reference ⚡ |
| :--------------------------- |
| In this section, we listed the references to the useful resources we discovered while working on this project. |

---

<a id="ref2"></a>
- LightGBM (Light Gradient Boosting Machine). (2021, December 22). GeeksforGeeks. Retrieved October 28, 2022, from https://www.geeksforgeeks.org/lightgbm-light-gradient-boosting-machine/

<a id="ref3"></a>
- What is LightGBM Algorithm, How to use it? (2020, June 25). Analytics Steps. Retrieved October 28, 2022, from https://www.analyticssteps.com/blogs/what-light-gbm-algorithm-how-use-it

<a id="nine"></a>
## 9. Acknowledgement
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Reference ⚡ |
| :--------------------------- |
| In this section, we acknowledged the contributions of people who made this project successful. |

---

In [ ]:
from datetime import date
today = date.today()
project_name = "Developing an Opinionated Sales Forecasting Model"
contributors = {
                'Kwazi Mbetu':"Supervisor",
                'John Mohale': "Supervisor",
                'Paul dos Santos': "Technical Consultant",
                'Muzi Xaba': "System Admin",
                'Zintle Faltein': "Head Teacher",
                'Andile Skosana': "System Admin"
               }

for names, role in zip(contributors.keys(), contributors.values()):
    print("""
Dear {},
            An Open Appreciation from EDSA Internship Group Team 28
    We really acknowledge and appreciate your contributions towards the success of
our project: {}.
    Your role as a {} really helped us scale through the project. You were there
for us when we needed you. Keep doing amazing things.

Regards,
Josh (for Team 28)
{}
    """.format(names,project_name, role, today))
    print("-"*90)